In [1]:
!pip install fastapi uvicorn python-multipart Pillow torch torchvision
!pip install pyngrok

# final 

In [7]:
with open('main.py', 'w') as f:
    f.write('''
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import torch
from PIL import Image
import io
from torchvision import transforms

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Replace with specific domains in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load the TorchScript model
model = torch.jit.load("/kaggle/input/final_model/pytorch/default/1/model_final.pt")
device = torch.device("cpu")  # Force to CPU
model.to(device)
model.eval()

# Define the image preprocessing function
def get_transforms():
    return transforms.Compose([
        transforms.Resize((248, 248)),
        transforms.ToTensor(),
    ])

def preprocess_and_predict(model, img_tensor):
    img_tensor = img_tensor.unsqueeze(0).to(device)  # Add batch dimension

    # Compute mean and std
    mean = torch.mean(img_tensor)
    std = torch.std(img_tensor)

    # Make prediction
    with torch.no_grad():
        output = model(img_tensor, mean.unsqueeze(0).to(device), std.unsqueeze(0).to(device))
        predicted_class = output.argmax(dim=1).item()

    return predicted_class, mean.item(), std.item()

# Define the response model
class PredictionResponse(BaseModel):
    predicted_class: int
    mean: float
    std: float
    description: str

@app.post("/predict/", response_model=PredictionResponse)
async def predict_image(file: UploadFile = File(...)):
    # Read image file
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes)).convert('L')

    # Preprocess the image
    transform = get_transforms()
    img_tensor = transform(image)

    # Run the prediction
    predicted_class, mean, std = preprocess_and_predict(model, img_tensor)

    # Map predicted class to dementia type
    dementia_types = {0: "Mild Dementia", 1: "Moderate Dementia", 2: "Non Demented", 3: "Very Mild Dementia"}
    description = dementia_types.get(predicted_class, "Not Supported")

    return PredictionResponse(
        predicted_class=predicted_class,
        mean=mean,
        std=std,
        description=description
    )

# Add a root route
@app.get("/")
async def read_root():
    return {"message": "Welcome to the Dementia Prediction API. Use the /predict/ endpoint to upload images."} ''')


In [8]:
!ngrok authtoken 2nevpSdeWmLxFqPvAHHTPDgNpGk_7mo6QCtKpevEuVRkiyFT8

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [10]:
from pyngrok import ngrok

# Start ngrok tunnel on port 8000
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")


Public URL: NgrokTunnel: "https://bf9a-34-151-77-179.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!uvicorn main:app --host 0.0.0.0 --port 8000


INFO:     Started server process [108]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     197.52.81.105:0 - "GET / HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "OPTIONS /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "OPTIONS /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "OPTIONS /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "OPTIONS /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     197.43.34.183:0 - "POST /predict/ HTTP/1.1" 200 OK


In [ ]:
with open('app.py', 'w') as f:
    f.write('''
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
import torch
from PIL import Image
import io
from torchvision import transforms
from pyngrok import ngrok

app = FastAPI()

# Load the TorchScript model
model = torch.jit.load("/kaggle/input/final_model/pytorch/default/1/model_final.pt")
device = torch.device("cpu")  # Force to CPU
model.to(device)
model.eval()

# Define the image preprocessing function
def get_transforms():
    transform = transforms.Compose([
        transforms.Resize((248, 248)),
        transforms.ToTensor(),
    ])
    return transform

def preprocess_and_predict(model, img_tensor):
    img_tensor = img_tensor.unsqueeze(0).to(device)  # Add batch dimension

    # Compute mean and std
    mean = torch.mean(img_tensor)
    std = torch.std(img_tensor)

    # Make prediction
    with torch.no_grad():
        output = model(img_tensor, mean.unsqueeze(0).to(device), std.unsqueeze(0).to(device))
        predicted_class = output.argmax(dim=1).item()

    return predicted_class, mean.item(), std.item()

# Define the response model
class PredictionResponse(BaseModel):
    predicted_class: int
    mean: float
    std: float
    description: str

@app.post("/predict/", response_model=PredictionResponse)
async def predict_image(file: UploadFile = File(...)):
    # Read image file
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes)).convert('L')

    # Preprocess the image
    transform = get_transforms()
    img_tensor = transform(image)

    # Run the prediction
    predicted_class, mean, std = preprocess_and_predict(model, img_tensor)

    # Map predicted class to dementia type
    dementia_types = {0: "Mild Dementia", 1: "Moderate Dementia", 2: "Non Demented", 3: "Very Mild Dementia"}
    description = dementia_types.get(predicted_class, "Not Supported")

    return PredictionResponse(
        predicted_class=predicted_class,
        mean=mean,
        std=std,
        description=description
    )

# Add a root route
@app.get("/")
async def read_root():
    return {"message": "Welcome to the Dementia Prediction API. Use the /predict/ endpoint to upload images."}

# Start ngrok tunnel on port 8000
ngrok.set_auth_token("2nevpSdeWmLxFqPvAHHTPDgNpGk_7mo6QCtKpevEuVRkiyFT8")  # Replace with your actual auth token
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the app using command line, outside this script:
!uvicorn main:app --host 0.0.0.0 --port 8000
''') 

In [ ]:
with open('main.py', 'w') as f:
    f.write('''
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
import torch
from PIL import Image
import io
from torchvision import transforms

app = FastAPI()

# Load the TorchScript model
model = torch.jit.load("/kaggle/input/final_model/pytorch/default/1/model_final.pt")
device = torch.device("cpu")  # Force to CPU
model.to(device)
model.eval()

# Define the image preprocessing function
def get_transforms():
    transform = transforms.Compose([
        transforms.Resize((248, 248)),
        transforms.ToTensor(),
    ])
    return transform

def preprocess_and_predict(model, img_tensor):
    img_tensor = img_tensor.unsqueeze(0).to(device)  # Add batch dimension

    # Compute mean and std
    mean = torch.mean(img_tensor)
    std = torch.std(img_tensor)

    # Make prediction
    with torch.no_grad():
        output = model(img_tensor, mean.unsqueeze(0).to(device), std.unsqueeze(0).to(device))
        predicted_class = output.argmax(dim=1).item()

    return predicted_class, mean.item(), std.item()

# Define the response model
class PredictionResponse(BaseModel):
    predicted_class: int
    mean: float
    std: float
    description: str

@app.post("/predict/", response_model=PredictionResponse)
async def predict_image(file: UploadFile = File(...)):
    # Read image file
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes)).convert('L')

    # Preprocess the image
    transform = get_transforms()
    img_tensor = transform(image)

    # Run the prediction
    predicted_class, mean, std = preprocess_and_predict(model, img_tensor)

    # Map predicted class to dementia type
    dementia_types = {0: "Mild Dementia", 1: "Moderate Dementia", 2: "Non Demented", 3: "Very Mild Dementia"}
    description = dementia_types.get(predicted_class, "Not Supported")

    return PredictionResponse(
        predicted_class=predicted_class,
        mean=mean,
        std=std,
        description=description
    )

# Add a root route
@app.get("/")
async def read_root():
    return {"message": "Welcome to the Dementia Prediction API. Use the /predict/ endpoint to upload images."}



''')
    